In [1]:
# Clone into the EMIT Resources Repository to access to helpful visualization tools
#!git clone https://github.com/nasa/EMIT-Data-Resources.git
#!cd EMIT-Data-Resources/python/modules
#!cp EMIT-Data-Resources/python/modules/emit_tools.py /home/jupyter/HIR_EMIT/hyperspectral_image_reconstruction_EMIT.ipynb

In [1]:
# Install necessary Python libraries
!pip install torch --quiet
!pip install hvplot --quiet
!pip install netCDF4 --quiet
!pip install spectral --quiet
!pip install rasterio --quiet
!pip install rioxarray --quiet
!pip install cartopy geoviews --quiet
!pip install s3fs --quiet
!pip install gdal

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import torch as nn
import os
import numpy as np
import rioxarray
import xarray as xr
import holoviews as hv
import hvplot.xarray
import netCDF4 as nc
import math
import warnings
import sys
sys.path.append('/home/jupyter/HIR_EMIT/EMIT-Data-Resources/python/modules')

from emit_tools import emit_xarray

warnings.simplefilter('ignore')
hv.extension('bokeh')

In [3]:
file_path = "EMIT_DATASET/EMIT_L2A_RFL_001_20241028T045642_2430203_015.nc"
ds_nc = nc.Dataset(file_path)
ds = xr.open_dataset(file_path)

In [4]:
# Loading 'sensor_band_parameters' and 'location' groups into separate datasets in order to access any additional data dimensions we need (i.e. wavelengths and geographic details).
wvl = xr.open_dataset(file_path, group='sensor_band_parameters')
loc = xr.open_dataset(file_path, group='location')

# Set the group's data as coordinates to the main dataset (using the spatial variables of downtrack and crosstrack)
ds = ds.assign_coords({'downtrack': ds.downtrack.data,
                       'crosstrack': ds.crosstrack.data,
                       **wvl.variables,
                       **loc.variables})
# Switch dimensions of the dataset from bands to wavelength to provide easier access to our spectral data
ds = ds.swap_dims({'bands': 'wavelengths'})

In [5]:
# Non-Orthorectified Spectral Plot
# Plotting the Surface Reflectance at a specific location for our current purposes (downtrack=660, crosstrack=370), showcasing how the reflectance varies with respect to wavelength
example_spectrum = ds['reflectance'].sel(downtrack=660, crosstrack=370)
example_spectrum.hvplot.line(y='reflectance', x='wavelengths', color='black', frame_height=400, frame_width=600)


:Curve   [wavelengths]   (Surface Reflectance)

In [ ]:
# Filtering Noisy Water Absorption Bands
# Reducing the potential for noise in the plot by filtering out any reflectance value for which the 'good_wavelengths' variable is 0, setting that reflectance value to NaN; This improves the overall visual clarity of the resulting plot
ds['reflectance'].data[:, :, ds['good_wavelengths'].data == 0] = np.nan
filtered_spectrum = ds['reflectance'].sel(downtrack=660, crosstrack=370)
filtered_spectrum.hvplot.line(y='reflectance', x='wavelengths', color='black', frame_height=400, frame_width=600)

In [ ]:
# Spatial Plot of a Specific Band (for now, 850nm)
# Creating a visual representation (spatial plot) for a given wavelength; This allows us to find spatial plots of all wavelengths we wish to use for the scope of our project and create test, training, and validation sets for our Hyperspectral Reconstruction model (for the purposes of this course, we wish to focus solely on the CO2 absorption bands centered at 15, 4.3, 2.7, and 2 µm)
refl850 = ds.sel(wavelengths=850, method='nearest')
refl850.hvplot.image(cmap='viridis', aspect='equal', frame_width=720).opts(title=f"{refl850.wavelengths.values:.3f} {refl850.wavelengths.units}")

In [ ]:
ds_geo = emit_xarray(file_path, ortho=True)
ds_geo.reflectance.data[ds_geo.reflectance.data == -9999] = np.nan  # Mask invalid values

ds_geo.sel(wavelengths=850, method='nearest').hvplot.image(cmap='Viridis', geo=True, tiles='ESRI', alpha=0.8, frame_height=600).opts(
    title=f"Reflectance at {refl850.wavelengths.values:.3f} {refl850.wavelengths.units} (Orthorectified)"
)

In [ ]:
rgb = ds_geo.sel(wavelengths=[650, 560, 470], method='nearest')

def gamma_adjust(rgb_ds, bright=0.2):
    array = rgb_ds.reflectance.data
    gamma = math.log(bright) / math.log(np.nanmean(array))
    rgb_ds.reflectance.data = np.clip(np.power(array, gamma), 0, 1)
    return rgb_ds

rgb = gamma_adjust(rgb)

# Create spatial RGB map
map = rgb.hvplot.rgb(x='longitude', y='latitude', bands='wavelengths', aspect='equal', frame_height=500)

# Stream configuration for interactive point selection
POINT_LIMIT = 10
color_cycle = hv.Cycle('Category20')
colors = [color_cycle[i] for i in range(5)]
xmid, ymid = ds_geo.longitude.mean().item(), ds_geo.latitude.mean().item()
first_point = ([xmid], [ymid], [0])
points = hv.Points(first_point, vdims='id')
points_stream = hv.streams.PointDraw(data=points.columns(), source=points, drag=True, num_objects=POINT_LIMIT)

In [ ]:
# Update to show reflectance spectra interactively
def plot_spectra(selected_points):
    lines = []
    for i, pt in enumerate(selected_points):
        x, y = pt[0], pt[1]
        spectrum = ds_geo.sel(longitude=x, latitude=y, method='nearest')['reflectance']
        line = hv.Curve((ds_geo.wavelengths, spectrum), label=f"Point {i}").opts(color=colors[i % len(colors)])
        lines.append(line)
    return hv.Overlay(lines).opts(width=600, height=400, legend_position='right')

spectra_dmap = hv.DynamicMap(plot_spectra, streams=[points_stream])

# Display combined map and spectral plot
(map + spectra_dmap).cols(1)
